In [1]:
from astropy.io import fits
import numpy as np
import skimage.data
from skimage import img_as_float
from skimage.restoration import denoise_wavelet, cycle_spin
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.table import Table, Column, MaskedColumn, vstack

%matplotlib inline

rcParams['figure.figsize'] = [18., 15.]

In [7]:
tab_all = Table.read("../STARS/Halpha-MC-STAR-intraNuvens-allFilter.ecsv", format='ascii.ecsv')
tab_all

Field,ID,RA,DEC,X_r,Y_r,s2n_r_psf,s2n_J0660_psf,s2n_i_psf,CLASS_STAR_r,CLASS_STAR_i,u_psf,J0378_psf,J0395_psf,J0410_psf,J0430_psf,g_psf,J0515_psf,r_psf,J0660_psf,i_psf,J0861_psf,z_psf,e_u_psf,e_J0378_psf,e_J0395_psf,e_J0410_psf,e_J0430_psf,e_g_psf,e_J0515_psf,e_r_psf,e_J0660_psf,e_i_psf,e_J0861_psf,e_z_psf
str6,str21,float32,float32,float32,float32,float32,float32,float32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float64,float64
MC0123,iDR4_3_MC0123_0019411,52.221207,-73.46904,5284.57,5452.06,49.227272,72.2,51.57143,1,1,99.0,99.0,99.0,99.0,99.0,19.467147827148438,19.30514907836914,18.109869,17.460869,17.282377,16.61092758178711,16.810928344726562,99.0,99.0,99.0,99.0,99.0,0.04399999976158142,0.05999999865889549,0.022,0.015,0.021,0.026000000536441803,0.03200000151991844
MC0138,iDR4_3_MC0138_0037893,29.304684,-74.70848,8180.15,5360.64,60.166668,47.086956,30.083334,1,1,18.117040634155273,17.819040298461914,17.65604019165039,17.67299461364746,17.728994369506836,17.855993270874023,17.85699462890625,18.029572,17.865572,18.238148,18.530210494995117,18.46921157836914,0.02800000086426735,0.02500000037252903,0.035999998450279236,0.03099999949336052,0.032999999821186066,0.017000000923871994,0.03200000151991844,0.018,0.023,0.036,0.0689999982714653,0.08699999749660492
MC0137,iDR4_3_MC0137_0024680,26.933619,-74.55015,3374.5,6410.22,60.166668,63.705883,38.67857,1,1,18.07911491394043,17.87311553955078,17.749935150146484,17.744356155395508,17.79535675048828,17.9333553314209,17.892356872558594,18.042515,17.833515,18.267185,18.41725730895996,18.418256759643555,0.027000000700354576,0.026000000536441803,0.039000000804662704,0.03400000184774399,0.032999999821186066,0.014999999664723873,0.026000000536441803,0.018,0.017,0.028,0.06700000166893005,0.05900000035762787
MC0001,iDR4_3_MC0001_0029699,66.26251,-63.099346,9734.62,6556.66,45.125,36.1,31.852942,1,1,19.66575813293457,19.754758834838867,19.852758407592773,19.056446075439453,19.206445693969727,18.70244598388672,18.508445739746094,18.158424,17.832424,17.991661,17.650461196899414,17.602460861206055,0.14300000667572021,0.06599999964237213,0.11500000208616257,0.04600000008940697,0.04899999871850014,0.028999999165534973,0.028999999165534973,0.024,0.03,0.034,0.0560000017285347,0.04800000041723251
MC0001,iDR4_3_MC0001_0034593,68.58173,-62.763313,2836.04,8784.95,24.613636,34.935482,21.66,1,1,99.0,20.739038467407227,99.0,19.813039779663086,19.602039337158203,18.969039916992188,18.889039993286133,18.185894,17.756893,17.843475,17.389266967773438,17.364267349243164,99.0,0.14300000667572021,99.0,0.08500000089406967,0.06300000101327896,0.039000000804662704,0.04100000113248825,0.044,0.031,0.05,0.04699999839067459,0.041999999433755875
MC0083,iDR4_3_MC0083_0036648,62.379604,-70.5175,6397.95,8441.15,57.0,54.15,41.653847,1,1,18.775346755981445,18.679346084594727,18.850345611572266,18.79446792602539,18.690467834472656,18.64346694946289,99.0,18.206917,17.842916,18.079536,18.004175186157227,18.017175674438477,0.027000000700354576,0.029999999329447746,0.03999999910593033,0.04100000113248825,0.03200000151991844,0.020999999716877937,99.0,0.019,0.02,0.026,0.039000000804662704,0.03799999877810478
MC0138,iDR4_3_MC0138_0013139,29.004818,-74.66809,8705.85,5610.06,49.227272,57.0,27.075,1,1,18.15501594543457,17.811016082763672,17.782804489135742,17.650989532470703,17.715896606445312,17.877897262573242,17.929990768432617,18.112976,18.019707,18.374174,18.488330841064453,18.549331665039062,0.026000000536441803,0.027000000700354576,0.03500000014901161,0.029999999329447746,0.029999999329447746,0.017000000923871994,0.02800000086426735,0.022,0.019,0.04,0.0729999989271164,0.0820000022649765
MC0099,iDR4_3_MC0099_0023403,35.76611,-72.213974,5079.52,5536.38,21.235294,24.613636,11.164948,1,1,19.29827308654785,18.705272674560547,18.67927360534668,18.42574691772461,18.41274642944336,18.17574691772461

In [11]:
tab_selected = Table.read("../STARS/object-to-observed-selected-all.dat", format='ascii')
tab_selected

32

In [9]:
# Making mask and applying
id1 = tab_all["ID"]
id2 = tab_selected["ID"]
mask = np.array([source in id2 for source in id1])

In [18]:
tab_all[mask]

Field,ID,RA,DEC,X_r,Y_r,s2n_r_psf,s2n_J0660_psf,s2n_i_psf,CLASS_STAR_r,CLASS_STAR_i,u_psf,J0378_psf,J0395_psf,J0410_psf,J0430_psf,g_psf,J0515_psf,r_psf,J0660_psf,i_psf,J0861_psf,z_psf,e_u_psf,e_J0378_psf,e_J0395_psf,e_J0410_psf,e_J0430_psf,e_g_psf,e_J0515_psf,e_r_psf,e_J0660_psf,e_i_psf,e_J0861_psf,e_z_psf
str6,str21,float32,float32,float32,float32,float32,float32,float32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float64,float64
MC0118,iDR4_3_MC0118_0002512,30.696186,-74.09912,1421.11,1250.01,33.84375,36.1,19.339285,1,1,18.70958709716797,18.13758659362793,17.969587326049805,17.9464168548584,17.99041748046875,18.112417221069336,18.06741714477539,18.28443,18.14543,18.536026,18.611148834228516,18.65414810180664,0.032999999821186066,0.035999998450279236,0.04600000008940697,0.04399999976158142,0.032999999821186066,0.028999999165534973,0.03799999877810478,0.032,0.03,0.056,0.06499999761581421,0.09799999743700027
MC0061,iDR4_3_MC0061_0007343,68.70252,-70.26852,2296.81,1777.3,24.613636,25.186047,26.414635,1,1,99.0,99.0,99.0,99.0,99.0,19.308841705322266,19.02684211730957,18.423212,17.900211,18.070469,17.95925521850586,17.86525535583496,99.0,99.0,99.0,99.0,99.0,0.05999999865889549,0.06599999964237213,0.044,0.043,0.041,0.06599999964237213,0.06400000303983688
MC0137,iDR4_3_MC0137_0038149,26.77703,-74.20252,3606.79,8690.57,34.935482,67.6875,37.344826,1,1,18.0707950592041,18.059795379638672,18.341794967651367,18.548696517944336,18.536697387695312,18.56669807434082,18.75569725036621,18.540657,17.718657,18.328768,18.326719284057617,18.32971954345703,0.02199999988079071,0.02800000086426735,0.057999998331069946,0.054999999701976776,0.052000001072883606,0.024000000208616257,0.04600000008940697,0.031,0.016,0.029,0.061000000685453415,0.05999999865889549
MC0137,iDR4_3_MC0137_0015372,27.26917,-74.813126,2834.59,4675.81,41.653847,57.0,30.083334,1,1,18.31709098815918,18.163089752197266,18.148090362548828,18.608413696289062,18.623931884765625,18.71541404724121,18.7694149017334,18.577227,17.919228,18.53591,18.171794891357422,18.37479591369629,0.02800000086426735,0.03200000151991844,0.0430000014603138,0.0560000017285347,0.052000001072883606,0.026000000536441803,0.032999999821186066,0.026,0.019,0.036,0.054999999701976776,0.05999999865889549
MC0078,iDR4_3_MC0078_0012417,42.87551,-71.00177,3337.72,5256.25,34.935482,36.1,26.414635,1,1,18.973731994628906,18.826732635498047,19.04073143005371,19.245193481445312,19.170194625854492,19.136194229125977,19.165193557739258,18.766077,18.453077,18.693459,18.645376205444336,18.674375534057617,0.039000000804662704,0.0430000014603138,0.07199999690055847,0.06800000369548798,0.06300000101327896,0.028999999165534973,0.041999999433755875,0.031,0.03,0.041,0.06800000369548798,0.0689999982714653
MC0021,iDR4_3_MC0021_0043791,69.88197,-66.09994,7548.32,3938.88,33.84375,13.884615,28.5,1,1,99.0,99.0,99.0,99.0,99.0,19.33926773071289,19.355268478393555,18.804817,18.327818,18.584064,18.602949142456055,18.595949172973633,99.0,99.0,99.0,99.0,99.0,0.04699999839067459,0.11400000005960464,0.032,0.078,0.038,0.08100000023841858,0.08100000023841858
MC0001,iDR4_3_MC0001_0015707,66.98884,-63.463417,7557.66,4209.46,49.227272,27.76923,38.67857,1,1,19.005226135253906,19.020225524902344,19.15322494506836,19.287050247192383,19.23805046081543,19.2220516204834,19.18505096435547,18.868732,18.542732,18.805666,18.917695999145508,18.936695098876953,0.07800000160932541,0.03200000151991844,0.04800000041723251,0.04600000008940697,0.03999999910593033,0.023000000044703484,0.03799999877810478,0.022,0.039,0.028,0.0949999988079071,0.09600000083446503
MC0146,iDR4_3_MC0146_0011885,69.397156,-75.16392,9861.8,2318.79,20.826923,27.76923,14.25,1,1,19.551969528198242,19.071969985961914,18.749969482421875,18.641626358032227,18.733627319335938,18.766626358032227,18.717626571655273,18.830908,18.344908,19.039501,18.98736572265625,19

In [15]:
# Save the final file (ASCII)
tab_all[mask].write("../STARS/Halpha-emitters-splus-selected.dat", format="ascii", overwrite=True)
